In [1]:
import pandas as pd
import pickle
import csv
from scipy import io as sio
import copy
from scipy.sparse import vstack

import numpy as np
from sklearn import svm
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
import random
from scipy.io import mmread
import math
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine
import os
from matplotlib import gridspec
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectKBest, SelectFromModel
from sklearn.feature_selection import chi2,f_classif,mutual_info_classif,mutual_info_regression
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from scipy.stats import chisquare
from scipy.stats import pearsonr
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import accuracy_score
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import time
import time
from scipy import io as sio
from scipy import sparse as ss
from scipy import optimize as so
from sklearn.utils.sparsefuncs import mean_variance_axis
from scipy.stats import linregress
import csv
import seaborn as sns
import copy
from sklearn.cluster import KMeans
import pickle
import numpy as np
import pandas as pd

import numpy as np
import random
import math
import os
import time

from sklearn import svm
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import normalize 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
plt.switch_backend('agg')
from matplotlib import gridspec

import pandas as pd
import pickle
from scipy.sparse import vstack

import os, psutil
import resource
import scanpy as sc
from sklearn.model_selection import KFold,GridSearchCV
import mat73

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 


## Six comparing Methods

In [4]:
class TimerError(Exception):
     """A custom exception used to report errors in use of Timer class"""

class Timer:
    def __init__(self):
        self._start_time = None

    def start(self):
        if self._start_time is not None:
            raise TimerError(f"Timer is running. Use .stop() to stop it")

        self._start_time = time.perf_counter()

    def stop(self):
        if self._start_time is None:
            raise TimerError(f"Timer is not running. Use .start() to start it")

        elapsed_time = time.perf_counter() - self._start_time
        self._start_time = None
        #print(f"Elapsed time: {elapsed_time:0.4f} seconds")
        return elapsed_time
        
def text_create(path, name, msg):
    full_path = path + "/" + name + '.pickle'
    f=open(full_path,'wb') 
    pickle.dump(msg,f)
    f.close()
    
def SVM(X, y,class_weight=None):
    model = svm.LinearSVC(loss='squared_hinge',tol=1, C=10,class_weight=class_weight, max_iter=1000)
    model.fit(X, y)
    return model

def get_error(model, X, y,sample_weight=None):
    y_pred = model.predict(X)
    return mean_squared_error(y_pred, y,sample_weight=sample_weight)


def compare_mincom_samples(path, memorys, X, Y, X_test,Y_test,num_features,init_samples,num_samples,class_weight=None):
    train_error=[]
    test_error=[]
    train_acc=[]
    test_acc=[]
    step_times=[]
    elapsed_times=[]
    D = np.shape(X)[1]
    N = np.shape(X)[0]
    
    if class_weight=='balanced':
        classes, inverse, count=np.unique(Y,return_inverse=True, return_counts=True)
        train_sample_weight=(Y.shape[0]/(len(classes)*count))[inverse]
        classes, inverse, count=np.unique(Y_test,return_inverse=True, return_counts=True)
        test_sample_weight=(Y_test.shape[0]/(len(classes)*count))[inverse]
    else:
        train_sample_weight=None
        test_sample_weight=None
    class1=[np.argwhere(Y==1)[0][0]]
    ind_all=[]
    ind_all.append(random.sample(range(N),init_samples-1)+class1)
    for i in range(num_features-1):
        ind_all.append(random.sample(range(N),num_samples-1)+class1)

    # Random
    memorys.append(psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2)
    memorys.append(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/ 1024 ** 2)
    et=Timer()
    et.start()
    ran_idx = random.sample(range(D), num_features)
    for i in range(num_features):
        t=Timer()
        t.start()
        ind=ind_all[i]
        model=SVM(X[:,ran_idx[:(i+1)]],Y,class_weight=class_weight)
        train_error.append(get_error(model,X[:,ran_idx[:(i+1)]],Y,sample_weight=train_sample_weight))
        train_acc.append(model.score(X[:,ran_idx[:(i+1)]],Y,sample_weight=train_sample_weight))
        test_error.append(get_error(model,X_test[:,ran_idx[:(i+1)]],Y_test,sample_weight=test_sample_weight))
        test_acc.append(model.score(X_test[:,ran_idx[:(i+1)]],Y_test,sample_weight=test_sample_weight))
        step_times.append(t.stop())
    elapsed_times.append(et.stop())
    print('ran_idx')
    
    # correlation coefficient
    memorys.append(psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2)
    memorys.append(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/ 1024 ** 2)
    et=Timer()
    et.start()
    cor_idx=[]
    for i in range(num_features):
        t=Timer()
        t.start()
        ind=ind_all[i]
        model = SelectKBest(f_classif, k=(i+1)).fit(X[ind],Y[ind])
        tmp=model.get_support(indices=True).tolist()
        cor_idx.append(tmp)
        model=SVM(X[:,tmp],Y,class_weight=class_weight)
        train_error.append(get_error(model,X[:,tmp],Y,sample_weight=train_sample_weight))
        train_acc.append(model.score(X[:,tmp],Y,sample_weight=train_sample_weight))
        test_error.append(get_error(model,X_test[:,tmp],Y_test,sample_weight=test_sample_weight))
        test_acc.append(model.score(X_test[:,tmp],Y_test,sample_weight=test_sample_weight))
        step_times.append(t.stop())
    elapsed_times.append(et.stop())
    print('cor_idx')
    
    # mutual
    memorys.append(psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2)
    memorys.append(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/ 1024 ** 2)
    et=Timer()
    et.start()
    mut_idx=[]
    for i in range(num_features):
        t=Timer()
        t.start()
        ind=ind_all[i]
        model = SelectKBest(mutual_info_classif, k=(i+1)).fit(X[ind],Y[ind])
        tmp=model.get_support(indices=True).tolist()
        mut_idx.append(tmp)
        model=SVM(X[:,tmp],Y,class_weight=class_weight)
        train_error.append(get_error(model,X[:,tmp],Y,sample_weight=train_sample_weight))
        train_acc.append(model.score(X[:,tmp],Y,sample_weight=train_sample_weight))
        test_error.append(get_error(model,X_test[:,tmp],Y_test,sample_weight=test_sample_weight))
        test_acc.append(model.score(X_test[:,tmp],Y_test,sample_weight=test_sample_weight))
        step_times.append(t.stop())
    elapsed_times.append(et.stop())
    print('mut_idx')
    
    # chi2
    memorys.append(psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2)
    memorys.append(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/ 1024 ** 2)
    et=Timer()
    et.start()
    chi_idx=[]
    for i in range(num_features):
        t=Timer()
        t.start()
        ind=ind_all[i]
        model = SelectKBest(chi2, k=(i+1)).fit(X[ind],Y[ind])
        tmp=model.get_support(indices=True).tolist()
        chi_idx.append(tmp)
        model=SVM(X[:,tmp],Y,class_weight=class_weight)
        train_error.append(get_error(model,X[:,tmp],Y,sample_weight=train_sample_weight))
        train_acc.append(model.score(X[:,tmp],Y,sample_weight=train_sample_weight))
        test_error.append(get_error(model,X_test[:,tmp],Y_test,sample_weight=test_sample_weight))
        test_acc.append(model.score(X_test[:,tmp],Y_test,sample_weight=test_sample_weight))
        step_times.append(t.stop())
    elapsed_times.append(et.stop())
    print('chi_idx')

    # tree
    memorys.append(psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2)
    memorys.append(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/ 1024 ** 2)
    et=Timer()
    et.start()
    t=Timer()
    t.start()
    tree_idx=[]
    for i in range(num_features):
        t=Timer()
        t.start()
        ind=ind_all[i]
        #model = ExtraTreesClassifier(criterion=para['criterion'],min_samples_leaf=para['min_samples_leaf']).fit(X[ind],Y[ind])
        model = ExtraTreesClassifier(min_samples_leaf=10).fit(X[ind],Y[ind])
        coef = [float('-inf') if math.isnan(x) or x==float('inf') else x for x in np.abs(model.feature_importances_)]
        tmp=sorted(range(len(coef)), key=lambda i: coef[i], reverse=True)[:(i+1)]
        tree_idx.append(tmp)
        model=SVM(X[:,tmp],Y,class_weight=class_weight)
        train_error.append(get_error(model,X[:,tmp],Y,sample_weight=train_sample_weight))
        train_acc.append(model.score(X[:,tmp],Y,sample_weight=train_sample_weight))
        test_error.append(get_error(model,X_test[:,tmp],Y_test,sample_weight=test_sample_weight))
        test_acc.append(model.score(X_test[:,tmp],Y_test,sample_weight=test_sample_weight))
        step_times.append(t.stop())
    elapsed_times.append(et.stop())
    print('tree_idx')
    
    # naive SVM
    memorys.append(psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2)
    memorys.append(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/ 1024 ** 2)
    et=Timer()
    et.start()
    t=Timer()
    t.start()
    svm_idx=[]
    for i in range(num_features):
        t=Timer()
        t.start()
        ind=ind_all[i]
        model=svm.LinearSVC(loss='squared_hinge',tol=1,C=10,class_weight=class_weight, max_iter=10000)
        model.fit(X[ind],Y[ind])
        weight=np.sum(np.abs(model.coef_),axis=0).tolist()
        coef = [0 if math.isnan(x) or x==float('inf') else x for x in weight]
        tmp=sorted(range(len(coef)), key=lambda i: coef[i], reverse=True)[:(i+1)]
        svm_idx.append(tmp)
        model=SVM(X[:,tmp],Y,class_weight=class_weight)
        train_error.append(get_error(model,X[:,tmp],Y,sample_weight=train_sample_weight))
        train_acc.append(model.score(X[:,tmp],Y,sample_weight=train_sample_weight))
        test_error.append(get_error(model,X_test[:,tmp],Y_test,sample_weight=test_sample_weight))
        test_acc.append(model.score(X_test[:,tmp],Y_test,sample_weight=test_sample_weight))
        step_times.append(t.stop())
    elapsed_times.append(et.stop())
    print('svm_idx')
    
    
    memorys.append(psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2)
    memorys.append(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/ 1024 ** 2)
    
    text_create(path,'compare_train_error',train_error)
    text_create(path,'compare_test_error',test_error)
    text_create(path,'compare_train_acc',train_acc)
    text_create(path,'compare_test_acc',test_acc)
    text_create(path,'ran_idx',ran_idx)
    text_create(path,'cor_idx',cor_idx)
    text_create(path,'mut_idx',mut_idx)
    text_create(path,'chi_idx',chi_idx)
    text_create(path,'tree_idx',tree_idx)
    text_create(path,'svm_idx',svm_idx)
    text_create(path,'ind_all',ind_all)
    text_create(path,'compare_step_times',step_times)
    text_create(path,'compare_elapsed_times',elapsed_times)
    text_create(path,'compare_memorys',memorys)




## Main

In [3]:
label_df=pd.read_csv('./data/cortex_svz_cellcentroids.csv')
target=label_df['Field of View'].values
raw_data = np.genfromtxt('./data/cortex_svz_counts_cell_types.csv', delimiter=',').transpose()
raw_data[0,0]=3
print(raw_data.shape)
classes=np.unique(target).tolist()
keys=classes
gene=pd.read_csv('./data/genes.csv',header=None)[0].values

x=label_df['X'].values
y=label_df['Y'].values

nonzero_row_indice, _ = raw_data.nonzero()
unique_nonzero_indice = np.unique(nonzero_row_indice)
raw_data=raw_data[unique_nonzero_indice]
target=target[unique_nonzero_indice]
raw_data=raw_data.transpose()
nonzero_row_indice, _ = raw_data.nonzero()
unique_nonzero_indice = np.unique(nonzero_row_indice)
raw_data=raw_data[unique_nonzero_indice]
gene=gene[unique_nonzero_indice]

data=normalize(raw_data.transpose(),axis=1, norm='l2')

idx = np.arange(np.shape(data)[0])
random.shuffle(idx)
X_train = data[idx[:int(np.shape(data)[0]*4/5)],:]
y_train = target[idx[:int(np.shape(data)[0]*4/5)]]
X_test = data[idx[int(np.shape(data)[0]*4/5):],:]
y_test = target[idx[int(np.shape(data)[0]*4/5):]]

del label_df,raw_data,nonzero_row_indice,unique_nonzero_indice

print(type(data))
print(keys)
print(np.shape(data),np.shape(target),len(np.unique(target)))
print(np.shape(X_train))
print(np.shape(X_test))
for i in np.unique(target):
    print(str(i)+' '+ str(keys[i])+': '+str(np.count_nonzero((target==i)*1)))

(913, 10000)
<class 'numpy.ndarray'>
[0, 1, 2, 3, 4, 5, 6]
(913, 10000) (913,) 7
(730, 10000)
(183, 10000)
0 0: 118
1 1: 108
2 2: 90
3 3: 106
4 4: 101
5 5: 281
6 6: 109


In [5]:
memorys=[]
memorys.append(psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2)
memorys.append(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/ 1024 ** 2)

'''
Parameters
'''
num_features = 30
init_samples=10
num_samples=10

path='results/compare_samecell'

try:
    os.mkdir('results')
except OSError:
    print ("Creation of the directory %s failed" % 'results')
else:
    print ("Successfully created the directory %s " % 'results')
try:
    os.mkdir(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)
    

compare_mincom_samples(path, memorys, X_train, y_train, X_test,y_test,num_features,init_samples=init_samples,
                       num_samples=num_samples,class_weight=None)




Creation of the directory results failed
Successfully created the directory results/compare_samecell 
ran_idx
cor_idx
mut_idx
chi_idx
tree_idx
svm_idx
